In [1]:
## YOLOv5-fc

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/siim/version5/
!ls

/content/drive/MyDrive/siim/version5
yolov5


In [8]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 9339, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 9339 (delta 14), reused 21 (delta 7), pack-reused 9309
Receiving objects: 100% (9339/9339), 9.75 MiB | 11.11 MiB/s, done.
Resolving deltas: 100% (6484/6484), done.
/content/drive/My Drive/siim/version5/yolov5
     |████████████████████████████████| 3.0 MB 2.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla T4)


In [5]:
# Access WANDB account
!pip install -q --upgrade wandb
 
import wandb
wandb.login()

wandb: Currently logged in as: jutraman (use `wandb login --relogin` to force relogin)


True

In [6]:
# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [7]:
TRAIN_PATH = "/content/drive/MyDrive/siim/fctmp/train/"
IMG_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 10

In [8]:
df = pd.read_csv("/content/drive/MyDrive/siim/summary/info_summary.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,image_id,study_id,pneumonia,pneumonia_class,height,width,boxes_x,image_label,image_label_id,filepath,data,id,boxes_y,label,StudyInstanceUID,image_level,fgfp,fcfp
0,0,0,b9175a64ad09,587699c00f43,Indeterminate Appearance,2,2618,3005,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity,1,/content/drive/MyDrive/siim/tmp/train/b9175a64...,train,b9175a64ad09_image,"[{'x': 2244.66714, 'y': 1575.60367, 'width': 2...",opacity 1 2244.66714 1575.60367 2504.06533 178...,587699c00f43,opacity,/content/drive/MyDrive/siim/fgtmp/train/b9175a...,/content/drive/MyDrive/siim/fctmp/train/b9175a...
1,1,1,8b2544f4361c,7d04273b5886,Typical Appearance,1,3480,4248,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity,1,/content/drive/MyDrive/siim/tmp/train/8b2544f4...,train,8b2544f4361c_image,"[{'x': 597.81637, 'y': 1014.4836, 'width': 515...",opacity 1 597.81637 1014.4836 1113.13638 2448....,7d04273b5886,opacity,/content/drive/MyDrive/siim/fgtmp/train/8b2544...,/content/drive/MyDrive/siim/fctmp/train/8b2544...
2,2,2,1808bdef30dd,7fd0efde20b7,Indeterminate Appearance,2,2336,2836,"[{'x': 566.36272, 'y': 480.19347, 'width': 483...",opacity,1,/content/drive/MyDrive/siim/tmp/train/1808bdef...,train,1808bdef30dd_image,"[{'x': 566.36272, 'y': 480.19347, 'width': 483...",opacity 1 566.36272 480.19347 1049.38085 1415....,7fd0efde20b7,opacity,/content/drive/MyDrive/siim/fgtmp/train/1808bd...,/content/drive/MyDrive/siim/fctmp/train/1808bd...
3,3,3,66c5884489fb,df21842cfe5e,Typical Appearance,1,2320,2832,"[{'x': 443.42472, 'y': 879.07294, 'width': 467...",opacity,1,/content/drive/MyDrive/siim/tmp/train/66c58844...,train,66c5884489fb_image,"[{'x': 443.42472, 'y': 879.07294, 'width': 467...",opacity 1 443.42472 879.07294 911.18833 1806.5...,df21842cfe5e,opacity,/content/drive/MyDrive/siim/fgtmp/train/66c588...,/content/drive/MyDrive/siim/fctmp/train/66c588...
4,4,4,95b239a10358,1d667e9556ad,Typical Appearance,1,2520,3032,"[{'x': 82.69968, 'y': 889.10112, 'width': 836....",opacity,1,/content/drive/MyDrive/siim/tmp/train/95b239a1...,train,95b239a10358_image,"[{'x': 82.69968, 'y': 889.10112, 'width': 836....",opacity 1 82.69968 889.10112 918.79295 2211.87...,1d667e9556ad,opacity,/content/drive/MyDrive/siim/fgtmp/train/95b239...,/content/drive/MyDrive/siim/fctmp/train/95b239...


In [12]:
os.makedirs('/content/drive/MyDrive/siim/version5/dataset/images/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/siim/version5/dataset/images/val', exist_ok=True)

os.makedirs('/content/drive/MyDrive/siim/version5/dataset/labels/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/siim/version5/dataset/labels/val', exist_ok=True)
print("Created folder structure")

Created folder structure


In [13]:
print(df.iloc[1,9],df.iloc[1,])

opacity Unnamed: 0                                                          1
Unnamed: 0.1                                                        1
image_id                                                 8b2544f4361c
study_id                                                 7d04273b5886
pneumonia                                          Typical Appearance
pneumonia_class                                                     1
height                                                           3480
width                                                            4248
boxes_x             [{'x': 597.81637, 'y': 1014.4836, 'width': 515...
image_label                                                   opacity
image_label_id                                                      1
filepath            /content/drive/MyDrive/siim/tmp/train/8b2544f4...
data                                                            train
id                                                 8b2544f4361c_image
boxes_y     

In [14]:
# Move the images to relevant split folder.
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    if row.data == 'train':
        copyfile(row.fcfp, f'/content/drive/MyDrive/siim/version5/dataset/images/train/{row.image_id}.jpg')
    else:
        copyfile(row.fcfp, f'/content/drive/MyDrive/siim/version5/dataset/images/val/{row.image_id}.jpg')

100%|██████████| 6334/6334 [37:27<00:00,  2.82it/s]


In [23]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '/content/drive/MyDrive/siim/version5/dataset/images/train',
    val = '/content/drive/MyDrive/siim/version5/dataset/images/val',
    nc = 2,
    names = ['none', 'opacity']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('/content/drive/MyDrive/siim/version5/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat /content/drive/MyDrive/siim/version5/yolov5/data/data.yaml

{names: [none, opacity], nc: 2, train: /content/drive/MyDrive/siim/version5/dataset/images/train,
  val: /content/drive/MyDrive/siim/version5/dataset/images/val}


In [24]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.width
    scale_y = IMG_SIZE/row.height
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [17]:
# Prepare the txt files for bounding box
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.image_id
    # Get split
    split = row.data
    # Get image-level label
    label = row.image_level
    
    if row.data=='train':
        file_name = f'/content/drive/MyDrive/siim/version5/dataset/labels/train/{row.image_id}.txt'
    else:
        file_name = f'/content/drive/MyDrive/siim/version5/dataset/labels/val/{row.image_id}.txt'
        
    
    if label=='opacity':
        # Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row, bboxes)
        # Format for YOLOv5
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

100%|██████████| 6334/6334 [00:20<00:00, 313.86it/s]


In [18]:
%cd /content/drive/MyDrive/siim/version5/yolov5/


/content/drive/MyDrive/siim/version5/yolov5


In [26]:
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project version5

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=version5, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2021-9-16 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspe

In [33]:
TEST_PATH = "/content/drive/MyDrive/siim/version5/dataset/images/val"
BEST_MODEL_PATH  ="/content/drive/MyDrive/siim/version5/yolov5/version5/exp3/weights/best.pt"

In [34]:
!python detect.py --weights {BEST_MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img 256 \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf

detect: weights=['/content/drive/MyDrive/siim/version5/yolov5/version5/exp3/weights/best.pt'], source=/content/drive/MyDrive/siim/version5/dataset/images/val, imgsz=[256, 256], conf_thres=0.281, iou_thres=0.5, max_det=3, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 2021-9-16 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPs
image 1/1267 /content/drive/MyDrive/siim/version5/dataset/images/val/0023f02ae886.jpg: 256x256 Done. (0.006s)
image 2/1267 /content/drive/MyDrive/siim/version5/dataset/images/val/004cbd797cd1.jpg: 256x256 2 opacitys, Done. (0.006s)
image 3/1267 /content/drive/MyDrive/siim/version5/dataset/images/val/006e2ac85d57.jpg: 256x256 1 opacity, Done

In [35]:
PRED_PATH = "runs/detect/exp4/labels/"
PRED_FILES = os.listdir(PRED_PATH)

print("Sample prediction (in txt file) : \n")

with open(PRED_PATH + PRED_FILES[0], "r") as f:
    ls = f.read().strip("\n").split(" ")
    print(f"LABEL : {ls[0]} \nX_CENTER : {ls[1]} \nY_CENTER : {ls[2]} \nWIDTH : {ls[3]} \nHEIGHT : {ls[4]} \nCONFIDENCE : {ls[5]}")

Sample prediction (in txt file) : 

LABEL : 1 
X_CENTER : 0.695312 
Y_CENTER : 0.574219 
WIDTH : 0.195312 
HEIGHT : 0.429688 
CONFIDENCE : 0.349254
1
